
<h1 style="text-align: center;">CIÊNCIA DE DADOS</h1>
<h1 style="text-align: center;">Roteiro de Atividade Prática</h1>
<br>
<br>

Nome: ______________________________________________________________________________________      

Turma: ______________


**Componente:** Inteligência Artificial
<br>
**Unidade Curricular:** IA Generativa
<br>
**Tema da Semana:** Implantação de LLMs
<br>


# Aula 1: Introdução à implantação de LLMs

## Tarefa
- Executem o código abaixo.

- Se as bibliotecas não tiverem instaladas, instale-as.

- Acompanhem as instruções.

- Observem a execução e discutam com seus colegas e professor.


### Atualizar bibliotecas


In [ ]:
!pip install -U langchain langchain-huggingface langchain-community transformers faiss-cpu


### Importar bibliotecas

In [ ]:

import os
from langchain.chains import RetrievalQA
from langchain_huggingface import HuggingFacePipeline
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import pipeline

## Criar embeddings para busca

In [ ]:

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


### Criar um arquivo com os textos desejados(para Políticas da Empresa, Regras de Segurança, etc..).

Nome do arquivo criado é "documento.txt" e foi adicionado ao mesmo a Política de Privacidade da Empresa XYZ.

**Neste exemplo iremos utilizar o texto em inglês, para isso utilize o google tradutor para adicionar mais informações**

In [ ]:

if not os.path.exists("dados"):
    os.makedirs("dados")

if not os.path.exists("dados/documento.txt"):
    with open("dados/documento.txt", "w", encoding="utf-8") as f:
        f.write("XYZ Company Privacy Policy\n\n"
            "1. Information Collection: XYZ Company collects personal information to improve its services.\n"
            "2. Use of Information: Data is used for customer support and recommendations.\n"
            "3. Data Sharing: We do not share information without consent.\n"
            "4. Security: We apply encryption to protect data.\n"
            "5. Updates: This policy may be changed and will be available on the website.")

print("Arquivo 'documento.txt' pronto!")



### Carregar documentos e criar vetores

In [ ]:
caminho_arquivo = "dados/documento.txt"
loader = TextLoader(caminho_arquivo)
documentos = loader.load()



### Dividir os documentos para indexação


In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
documentos_divididos = text_splitter.split_documents(documentos)



### Criar banco de vetores FAISS


In [ ]:

vectorstore = FAISS.from_documents(documentos_divididos, embeddings)
retriever = vectorstore.as_retriever()



### Criar modelo LLM leve


In [ ]:
llm_pipeline = pipeline(
    "text-generation",
    model="gpt2",  #  Modelo mais leve para evitar estouro de RAM
    max_new_tokens=100,  #  Ajuste do número de tokens gerados
    temperature=0.7,  #  Controla a criatividade do texto gerado
    pad_token_id=50256  #  Evita warnings de padding
)



#  Criar o modelo LangChain


In [ ]:
llm = HuggingFacePipeline(pipeline=llm_pipeline)



### Criar a cadeia de recuperação e geração de resposta (RAG)


In [ ]:
qa = RetrievalQA.from_llm(llm=llm, retriever=retriever)


### Testar a IA com recuperação de documentos

Aproveite para testar outras perguntas e compare as respostas.

In [ ]:
pergunta = "What is the company's privacy policy about security?"
resposta = qa.invoke({"query": pergunta})

print(f"\nPergunta/Query:\n", resposta['query'])
print(f"\nResultado:\n", resposta['result'])


In [ ]:
pergunta = "What the company's privacy policy says about security"
resposta = qa.invoke({"query": pergunta})

print(f"\nPergunta/Query:\n", resposta['query'])
print(f"\nResultado:\n", resposta['result'])
